In [ ]:
from train import train
from config import Config

class CFG(Config):
    base_model = 'efficientnet_b0'   # resnet18/34/50, efficientnet_b0/b1/b2/b3/b4, efficientnet_v2_s, convnext_tiny, swin_t
    batch_size = 16
    epochs = 3
    base_lr = 1e-3
    scheduler_step_size = 2
    optimizer = 'AdamW'
    scheduler = 'StepLR'
    loss = 'KLDivLoss'
    lr_gamma = 0.1
    sgd_momentum = 0.9
    random_erasing_p = 0
    freeze_epochs = 0
    spec_random_trial_num = 1
    eeg_random_trial_num = 1
    data_type = 'eeg_tf'  # 'spec', 'eeg_tf', 'spec+eeg_tf
    eeg_tf_data = 'eeg_tf_data_globalnorm'

    # Augmentation
    use_mixup = False
    mixup_alpha = 2.0
    xy_masking_args = dict(p=0.5, num_masks_x=(2, 6), num_masks_y=(2, 6), max_x_length=(40, 20), max_y_length=(10, 20), fill_value=0)

    if data_type == 'spec':
        in_channels = 1
        spec_trial_selection = 'first'
        eeg_trial_selection = 'all'
    elif data_type == 'eeg_tf':
        in_channels = 1
        spec_trial_selection = 'all'
        eeg_trial_selection = 'first'
    elif data_type == 'spec+eeg_tf':
        in_channels = 2
        spec_trial_selection = 'all'
        eeg_trial_selection = 'first'


for scheduler in ['StepLR']:
    CFG.scheduler = scheduler
    for epochs in [3]:
        CFG.epochs = epochs
        for eeg_tf_data in ['eeg_tf_data_globalnorm']:
            CFG.eeg_tf_data = eeg_tf_data
            for base_model in ['efficientnet_b0']:
                CFG.base_model = base_model
                for data_type in ['eeg_tf']:
                    CFG.data_type = data_type
                    for mixup_alpha in [0]:
                        CFG.mixup_alpha = mixup_alpha
                        for batch_size in [16]:
                            CFG.batch_size = batch_size
                            for xy_masking_args in [dict(p=0.5, num_masks_x=(2, 6), num_masks_y=0, mask_x_length=(40, 60), mask_y_length=0, fill_value=0)]:
                                CFG.xy_masking_args = xy_masking_args

                                # Setup and train model
                                if data_type == 'spec':
                                    CFG.in_channels = 1
                                    CFG.spec_trial_selection = 'first'
                                    CFG.eeg_trial_selection = 'all'
                                elif data_type == 'eeg_tf':
                                    CFG.in_channels = 1
                                    CFG.spec_trial_selection = 'all'
                                    CFG.eeg_trial_selection = 'first'
                                elif data_type == 'spec+eeg_tf':
                                    CFG.spec_trial_selection = 'all'
                                    CFG.eeg_trial_selection = 'first'
                                train(CFG)